In [1]:
#!pip install scikit-image

# 0. Loading Dataset

You can download the dataset from {https://darwin.v7labs.com/v7-labs/covid-19-chest-x-ray-dataset?sort=priority\%3Adesc}.
The data entitled as '`darwin dataset pull v7-labs/covid-19-chest-x-ray-dataset:all-images`' will be used in this assignment. All dataset consist of 6504 images from 702 classes. We will extract the images of 4 classes (Bacterial Pneumonia, Viral Pneumonia, No Pneumonia (healthy), Covid-19) and save them as .npy file with the following code:

In [2]:
'''
All packages and modules are imported in the "functions.py" file located next to this notebook

Additionally a number of code chunks employed in previous versions of this notebook have been
defined into functions and placed in that same file in order to simplify the exposition of the 
main points and results of the assignment. For further insight on the code behind the actions
here executed this "functions.py" file can be visited, there all the details can be found.

IMPORTANT: for the appropriate functioning of this notebook it must be placed in the same 
directory as "functions.py"
'''

from functions import *

2023-03-04 06:23:27.401136: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-04 06:23:28.609245: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-04 06:23:28.609312: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-04 06:23:32.272777: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [3]:
###################################
# > DISABLED - ALREADY EXECUTED < #
###################################


# # all-images file should be uploaded to the same file
# imageNames = glob.glob("all-images/*")
# 
# dataset = []
# labels = []
# 
# for i, imName in enumerate(imageNames):
# 
#     # Opening JSON file
#     f = open(imName)
#     data = json.load(f)
#     for j in range(len(data['annotations'])):
# 
#         if 'COVID-19' in (data['annotations'][j]['name']):
#           #load images from url    
#             urllib.request.urlretrieve(data['image']['url'],"img.png")    
#             img = Image.open("img.png")
#             #convert images to grayscale
#             imgGray = img.convert('L')
#             #resize the image (156x156)
#             im = imgGray.resize((156,156), Image.LANCZOS)           
#             label = data['annotations'][j]['name']
#             dataset.append(np.array(im))
#             labels.append(label)
#             print(label)
#             break
# 
#         if 'Viral Pneumonia' in (data['annotations'][j]['name']) \
#             or 'Bacterial Pneumonia' in (data['annotations'][j]['name']) \
#             or 'No Pneumonia (healthy)' in (data['annotations'][j]['name']):
#             #load images from url    
#             urllib.request.urlretrieve(data['image']['url'],"img.png")    
#             img = Image.open("img.png")
#             #convert images to grayscale
#             imgGray = img.convert('L')
#             #resize the image (156x156)
#             im = imgGray.resize((156,156), Image.LANCZOS)           
#             label = data['annotations'][j]['name']
#             dataset.append(np.array(im))
#             labels.append(label)
#             break
# 
# #Convert data shape of (n_of_samples, width, height, 1)
# dataset = np.dstack(dataset)    
# dataset = np.rollaxis(dataset,-1)
# labels = np.array(labels)
# 
# #convert images gray scale to rgb
# data = np.array(layers.Lambda(tf.image.grayscale_to_rgb)(tf.expand_dims(dataset, -1)))
# 
# # save data and labels into a folder
# np.save("data.npy", data)
# np.save("labels.npy", labels)

Once you save your data, you can load it from your directory.

In [4]:
features = np.load('data/data.npy')
labels = np.load('data/labels.npy')

# 1. Data preprocessing



### 1.1 Splitting Data

In [5]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.2, random_state=42, stratify=labels)

# Further split the training set into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.25, random_state=42, stratify=y_train)


### 1.2 Normalize Data

In [6]:
# Float conversion to allow normalization
X_train=X_train.astype('float32')
X_test=X_test.astype('float32')
X_val=X_val.astype('float32')

# Normalization  
X_train=X_train/255.0
X_test=X_test/255.0
X_val=X_val/255.0

# Compute the mean and standard deviation of the training set for standardization (NOT USED)
train_mean = np.mean(X_train, axis=0)
train_std = np.std(X_train, axis=0)

# Standardization (NOT USED)
X_train_norm = (X_train - train_mean) / train_std
X_val_norm = (X_val - train_mean) / train_std
X_test_norm = (X_test - train_mean) / train_std


### 1.3 Categorical encoding

In [7]:
# Define a dictionary that maps each category to a numerical value
label_map = {"Bacterial Pneumonia": 0, "Viral Pneumonia": 1, "No Pneumonia (healthy)": 2, "COVID-19": 3}

# Encode the categorical labels as numerical values using the label map
y_train_encoded = np.vectorize(label_map.get)(y_train)
y_val_encoded = np.vectorize(label_map.get)(y_val)
y_test_encoded = np.vectorize(label_map.get)(y_test)

# Convert the numerical labels to one-hot encoded format
num_classes = 4
y_train_onehot = keras.utils.to_categorical(y_train_encoded, num_classes=num_classes)
y_val_onehot = keras.utils.to_categorical(y_val_encoded, num_classes=num_classes)
y_test_onehot = keras.utils.to_categorical(y_test_encoded, num_classes=num_classes)



# 2. Baseline Model

### 2.1 Create baseline model

In [13]:

from tensorflow import keras

def build_baseline_model():
    model = keras.Sequential([
        # Convolutional layers
        keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', input_shape=(156, 156, 3)),
        keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
        keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
        keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        
        # Dense layers
        keras.layers.Flatten(),
        keras.layers.Dense(32, activation='relu'),
        keras.layers.Dense(32, activation='relu'),
        keras.layers.Dense(4, activation='softmax')
    ])

    # Compile the model with appropriate loss function, optimizer, and metrics
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model


baseline_model = build_baseline_model()

# Train the model for 10 epochs with a batch size of 32
history = baseline_model.fit(
    X_train_norm,
    y_train_onehot,
    batch_size=32,
    epochs=10,
    validation_data=(X_val_norm, y_val_onehot)
)


Epoch 1/10
128/128 [==============================] - 645s 5s/step - loss: 0.7968 - accuracy: 0.6746 - val_loss: 0.6137 - val_accuracy: 0.7390
Epoch 2/10
128/128 [==============================] - 536s 4s/step - loss: 0.6372 - accuracy: 0.7391 - val_loss: 0.6423 - val_accuracy: 0.7312
Epoch 3/10
128/128 [==============================] - 473s 4s/step - loss: 0.5594 - accuracy: 0.7702 - val_loss: 0.5811 - val_accuracy: 0.7586
Epoch 4/10
128/128 [==============================] - 373s 3s/step - loss: 0.5050 - accuracy: 0.7939 - val_loss: 0.5608 - val_accuracy: 0.7615
Epoch 5/10
128/128 [==============================] - 374s 3s/step - loss: 0.4573 - accuracy: 0.8144 - val_loss: 0.6046 - val_accuracy: 0.7576
Epoch 6/10
128/128 [==============================] - 369s 3s/step - loss: 0.4232 - accuracy: 0.8262 - val_loss: 0.5628 - val_accuracy: 0.7889
Epoch 7/10
128/128 [==============================] - 359s 3s/step - loss: 0.3493 - accuracy: 0.8567 - val_loss: 0.5934 - val_accuracy: 0.7634

In [ ]:
baseline_all_val_acc = np.mean(history.history["val_accuracy"])
baseline_all_val_loss = np.mean(history.history["val_loss"])
baseline_all_train_acc = np.mean(history.history["accuracy"])
baseline_all_train_loss = np.mean(history.history["loss"])
print("BASELINE RESULTS:")
print("-"*len("BASELINE RESULTS:"))
print()
print("**Training**")
print("The average training accuracy among all epochs is: {:.4}".format(baseline_all_train_acc))
print("The average training loss among all epochs is: {:.4}".format(baseline_all_train_loss))
print()
print("**Validation**")
print("The average validation accuracy among all epochs is: {:.4}".format(baseline_all_val_acc))
print("The average validation loss among all epochs is: {:.4}".format(baseline_all_val_loss))

### 2.2 Analyze the performance of the baseline model

In [ ]:
import matplotlib.pyplot as plt

##Plot for the accuracy of the baseline model 
accuracy_train = history.history['accuracy']
accuracy_val = history.history['val_accuracy']
plt.plot(accuracy_train, label='training_accuracy')
plt.plot(accuracy_val, label='validation_accuracy')
plt.title('ACCURACY OF THE MODEL')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

##Plot for the loss of the baseline model 
loss_train = history.history['loss']
loss_val = history.history['val_loss']
plt.plot(loss_train, label='training_loss')
plt.plot(loss_val, label='validation_loss')
plt.title('LOSS OF MODEL')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

##ROC curve 
y_pred = baseline_model.predict(X_test_norm) 

from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
fpr = {}
tpr = {}
roc_auc = {}
#calculating roc for each class
for i in range(num_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_onehot[:,i], y_pred[:,i])
    roc_auc[i] = auc(fpr[i], tpr[i])
from sklearn.metrics import roc_auc_score
# calculating micro-average ROC curve and  area
fpr_micro, tpr_micro, _ = roc_curve(y_test_onehot.ravel(), y_pred.ravel())
roc_auc_micro = roc_auc_score(y_test_onehot.ravel(), y_pred.ravel())
# Compute macro-average ROC curve and  area
fpr_macro = np.unique(np.concatenate([fpr[i] for i in range(num_classes)]))
tpr_macro = np.zeros_like(fpr_macro)
for i in range(num_classes):
    tpr_macro += np.interp(fpr_macro, fpr[i], tpr[i])
tpr_macro /= num_classes
roc_auc_macro = auc(fpr_macro, tpr_macro)
#Plot the ROC curve for each class using matplotlib.pyplot.plot()
plt.figure(figsize=(10, 5))
lw = 2
for i in range(num_classes):
    plt.plot(fpr[i], tpr[i], lw=lw, label='ROC curve of class %d (area = %0.2f)' % (i, roc_auc[i]))
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.plot(fpr_micro, tpr_micro,lw=lw, linestyle='--', label='micro-average ROC curve (area = %0.2f)' % (roc_auc_micro))
plt.plot(fpr_macro, tpr_macro,lw=lw, linestyle='--', label='macro-average ROC curve (area = %0.2f)' % (roc_auc_macro))
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic of Multiclass')
plt.legend(loc="lower right")
plt.show()


#reversing pred to categorical so to get the labels 
inverse_label_map = {v: k for k, v in label_map.items()}  # invert the label_map
y_pred_decoded_numerical = np.argmax(y_pred, axis=1)
y_pred_decoded_categorical = np.vectorize(inverse_label_map.get)(y_pred_decoded_numerical)



#confusion matrix 
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred_decoded_categorical)
classes = np.unique(y_test)
# plot the confusion matrix
fig, ax = plt.subplots()
im = ax.imshow(cm, interpolation='nearest', cmap='Reds')
ax.figure.colorbar(im, ax=ax)
ax.set(xticks=np.arange(cm.shape[1]), yticks=np.arange(cm.shape[0]), xticklabels=classes, yticklabels=classes, ylabel='True label', xlabel='Predicted label')

# rotate the labels
plt.setp(ax.get_xticklabels(), rotation=20, ha="right", rotation_mode="anchor")
# text annotations like the numbers inside 
thresh = cm.max() / 2.
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        ax.text(j, i, format(cm[i, j], 'd'), ha="center", va="center", color="white" if cm[i, j] > thresh else "black")
plt.show()


#precision and recall and f1-score 
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_decoded_categorical))


# 3. Adapting/fine-tuning the network

In [8]:
#############################################
#             CONTROL PANEL

# Sample restriction
restriction = True
address_vp_imbalance = True
proportion = 0.25

# Sample augmentation
augmentation_flip = False                       
augmentation_rotate = False

#                                           
#############################################


# safety mechanism: augmentation and restriction are not meant to be combined
if restriction == True:
    agumentation_flip = False
    augmentation_rotate = False

### 3.1 Data restriction

In [10]:
if restriction == True:
    
    if address_vp_imbalance == True:
        restricted_sample_train, restricted_sample_labels_train = restrict_sample_vp_imb(proportion , X_train, y_train)
        
    else:
        restricted_sample_train, restricted_sample_labels_train = restrict_sample(proportion , X_train, y_train)

In [11]:
if restriction == True:
    
    ############################
    # > RESTRICTION  SUMMARY < #
    ############################
    print("SAMPLE RESTRICTION SUMMARY")
    print("")
    print("")
    print("ORIGINAL SAMPLE")
    print(F"Original number of instances: {len(X_train)}")
    print(f"Original instance distribution by class: \n {pd.Series(y_train).value_counts()}")
    print("")
    print("RESTRICTED SAMPLE")
    print(f"Number of instances in restricted sample: {len(restricted_sample)}")
    print(f"Instance distribution by class in restricted sample: \n {pd.Series(restricted_sample_labels).value_counts()}")
    
    X_train = np.array(restricted_sample)
    y_train = np.array(restricted_sample_labels)
    
else:
    print("SAMPLE RESTRICTION WAS NOT CONDUCTED")

SAMPLE RESTRICTION SUMMARY


ORIGINAL SAMPLE
Original number of instances: 4147
Original instance distribution by class: 
 Bacterial Pneumonia       1689
Viral Pneumonia           1182
No Pneumonia (healthy)     963
COVID-19                   313
dtype: int64

RESTRICTED SAMPLE
Number of instances in restricted sample: 957
Instance distribution by class in restricted sample: 
 Bacterial Pneumonia       422
COVID-19                  313
Viral Pneumonia           295
No Pneumonia (healthy)    240
dtype: int64


### 3.2 Data Augmentation

In [12]:
if augmentation_flip == True:

    augmentedfeatures, augmentedlabels_ = augment_sample(X_train, y_train)

In [13]:
if augmentation_flip == True:
    
    ############################
    # > AUGMENTATION SUMMARY < #
    ############################
    
    print("SAMPLE AUGMENTATION SUMMARY")
    print("")
    print("")
    print("ORIGINAL SAMPLE")
    print(F"Original number of instances: {len(labels)}")
    print(f"Original instance distribution by class: \n {pd.Series(labels).value_counts()}")
    print("")
    print("AUGMENTED SAMPLE")
    print(f"Number of instances in augmented sample: {len(augmentedlabels)}")
    print(f"Instance distribution by class in augmented sample: \n {pd.Series(augmentedlabels).value_counts()}")
    
    X_train = np.array(augmentedfeatures)
    y_train = np.array(augmentedlabels)

else:
    print("SAMPLE AUGMENTATION WAS NOT CONDUCTED")

SAMPLE AUGMENTATION WAS NOT CONDUCTED


### 3.3 Categorical encoding of new labels

In [14]:
## Define a dictionary that maps each category to a numerical value
#label_map = {"Bacterial Pneumonia": 0, "Viral Pneumonia": 1, "No Pneumonia (healthy)": 2, "COVID-19": 3}
#
## Encode the categorical labels as numerical values using the label map
y_train_encoded = np.vectorize(label_map.get)(y_train)
#y_val_encoded = np.vectorize(label_map.get)(y_val)
#y_test_encoded = np.vectorize(label_map.get)(y_test)
#
## Convert the numerical labels to one-hot encoded format
#num_classes = 4
y_train_onehot = keras.utils.to_categorical(y_train_encoded, num_classes=num_classes)
#y_val_onehot = keras.utils.to_categorical(y_val_encoded, num_classes=num_classes)
#y_test_onehot = keras.utils.to_categorical(y_test_encoded, num_classes=num_classes)

### 3.4 Network fine tuning

In [ ]:
#After some tryouts, Nadam reported an extra 0-2% val_accuracy in our baseline model vs Adam
#So we will find the best learning rate for this optimizer:

from tensorflow import keras
import keras.backend as K
import matplotlib.pyplot as plt

def lr_schedule(epoch, initial_lr, final_lr, total_epochs):
    """
    calculates the learning rate for each epoch based on the initial learning rate, final learning rate, and total number of epochs
    """
    lr = initial_lr + (final_lr - initial_lr) * (epoch / float(total_epochs))
    return lr

def plot_lr_schedule(initial_lr, final_lr, total_epochs):
    lr = [lr_schedule(epoch, initial_lr, final_lr, total_epochs) for epoch in range(total_epochs)]
    plt.plot(lr, history.history['val_loss'])
    plt.xlabel('Learning Rate')
    plt.ylabel('Validation Loss')
    plt.title('Learning Rate Schedule')
    plt.show()

# Adding a Lr Scheduler to check the learning rate evolution during training and to avoid overfitting
initial_lr = 0.001
final_lr = 0.01
baseline_epochs = 10

lr_scheduler = keras.callbacks.LearningRateScheduler(lambda epoch: lr_schedule(epoch, initial_lr, final_lr, baseline_epochs))

baseline_model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['accuracy'])

# Train the model for 4 epochs with a batch size of 32
history = baseline_model.fit(
    X_train_norm,
    y_train_onehot,
    batch_size=32,
    epochs=baseline_epochs,
    validation_data=(X_val_norm, y_val_onehot),
    callbacks=[lr_scheduler]
)



In [ ]:
# Rule of thumb: optimal will be a bit lower than when lr starts climbing, usually 10 times lower the climb up point (around 0.005)
plot_lr_schedule(initial_lr, final_lr, baseline_epochs)

In [ ]:
"""from keras import backend as K

# Some memory clean-up
K.clear_session()"""

In [1]:
# TUNED MODEL 1: BASELINE + LR: Nadam + KFOLD VALIDATION + EARLY STOPPING + LeakyReLU + Extra Dense layer + 3 Dropout layers + Doubled batch size to 64
# FAILED CHANGES VS BASELINE INDICATED WITH A HASHTAG

from tensorflow.keras.layers import LeakyReLU

tuned_m_batch = 64

def build_tuned_model():
    model = keras.Sequential([
    # Convolutional layers
    keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation=LeakyReLU(), input_shape=(156, 156, 3)),
    keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation=LeakyReLU()),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation=LeakyReLU()),
    keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation=LeakyReLU()),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation=LeakyReLU()),
    keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation=LeakyReLU()),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    # (FAILED CHANGE VS BASELINE) Adding another pack of Conv2D and MaxPooling2D layers
    #keras.layers.Conv2D(filters=64, kernel_size=(3, 3),  activation=keras.layers.LeakyReLU(alpha=0.01), kernel_regularizer=l2(0.01)),
    #keras.layers.Conv2D(filters=32, kernel_size=(3, 3),  activation=keras.layers.LeakyReLU(alpha=0.01), kernel_regularizer=l2(0.01)),
    #keras.layers.MaxPooling2D(pool_size=(2, 2)),
    # Dense layers
    keras.layers.Flatten(),
    keras.layers.Dense(32, activation=LeakyReLU()),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(32, activation=LeakyReLU()),
    keras.layers.Dropout(0.2),    
    keras.layers.Dense(32, activation=LeakyReLU()),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(4, activation='softmax')])
    # Compile the model with appropriate loss function, optimizer, and metrics
    # (FAILED CHANGE VS BASELINE) Adding the optimal using lr schedule for SGD
    optim = keras.optimizers.Nadam(learning_rate=0.001) #(FAILED CHANGE VS BASELINE) optimizer = adam, sgd, rmsprop

    model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy'])
    return model

2023-03-05 12:15:56.749145: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-05 12:15:59.214801: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-05 12:15:59.443697: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-05 12:15:59.443875: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

def build_h_model():
    '''restnet blocks'''
    input_shape = (156, 156, 3)
    inputs = keras.Input(shape=input_shape)
     # Convolutional layers
    x = keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation=keras.layers.LeakyReLU(alpha=0.01),  padding='same')(inputs)
    x = keras.layers.Conv2D(filters=32, kernel_size=(3, 3),  activation=keras.layers.LeakyReLU(alpha=0.01), padding='same')(x)
    x = keras.layers.MaxPooling2D(pool_size=(2, 2))(x)

    #  block 1
    out=x
    x = keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation=keras.layers.LeakyReLU(alpha=0.1), padding='same')(x)
    x = keras.layers.Conv2D(filters=32, kernel_size=(3, 3),  activation=keras.layers.LeakyReLU(alpha=0.1), padding='same')(x)
    x = keras.layers.add([out, x])
    x = keras.layers.LeakyReLU(alpha=0.1)(x)

    #  block 2
    out=x
    x = keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation=keras.layers.LeakyReLU(alpha=0.1), padding='same')(x)
    x = keras.layers.Conv2D(filters=32, kernel_size=(3, 3),  activation=keras.layers.LeakyReLU(alpha=0.1), padding='same')(x)
    x = keras.layers.add([out, x])
    x = keras.layers.LeakyReLU(alpha=0.1)(x)
    #  block 3
    out=x
    x = keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation=keras.layers.LeakyReLU(alpha=0.1), padding='same')(x)
    x = keras.layers.Conv2D(filters=32, kernel_size=(3, 3),  activation=keras.layers.LeakyReLU(alpha=0.1), padding='same')(x)
    x = keras.layers.add([out, x])
    x = keras.layers.LeakyReLU(alpha=0.1)(x)
    # Dense layers
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(32, activation=keras.layers.LeakyReLU(alpha=0.1))(x)
    x = keras.layers.Dense(32, activation=keras.layers.LeakyReLU(alpha=0.1))(x)
    outputs = keras.layers.Dense(4, activation='softmax')(x)
    optim = keras.optimizers.Nadam(learning_rate=0.001) 


    # Create model
    model = keras.Model(inputs=inputs, outputs=outputs, name='h_model')

    # Compile the model with appropriate loss function, optimizer, and metrics
    model.compile(loss='categorical_crossentropy', optimizer=optim, metrics=['accuracy'])
    return model


hybrid_model = build_h_model()

# Train the model for 10 epochs with a batch size of 32
'''history = hybrid_model.fit(
    X_train_norm,
    y_train_onehot,
    batch_size=32,
    epochs=10,
    validation_data=(X_val_norm, y_val_onehot))'''


In [ ]:
# K FOLD VALIDATION (5 max epochs for speeding purposes)

from sklearn.model_selection import KFold

k = 4
num_val_samples = len(X_train_norm) // k 
num_epochs = 5
tuned_all_val_losses = [] # Should add the score of each run at the end of the loop
tuned_all_val_acc = []
base_all_val_losses = []
base_all_val_acc = []
res_all_val_acc=[]
res_all_val_losses=[]

for i in range(k):
    print('processing fold #', i)
    # Prepare the validation data: data from partition # k
    val_data = X_train_norm[i * num_val_samples: (i + 1) * num_val_samples] 
    val_targets = y_train_onehot[i * num_val_samples: (i + 1) * num_val_samples]

    # Prepare the training data: data from all other partitions
    partial_train_data = np.concatenate(
        [X_train_norm[:i * num_val_samples],
         X_train_norm[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [y_train_onehot[:i * num_val_samples],
         y_train_onehot[(i + 1) * num_val_samples:]],
        axis=0)
    # (CHANGE VS BASELINE)Defining EarlyStopping callback
    early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)

    baseline_model = build_baseline_model()
    tuned_model_1 = build_tuned_model()
    hybrid_model = build_h_model()
    

    # Train baseline and tuned models for 4 epochs with a batch size of 32
    print('processing baseline model')
    baseline_history = baseline_model.fit(
        partial_train_data,
        partial_train_targets,
        batch_size=32,
        epochs=num_epochs,
        validation_data=(val_data, val_targets),
    )
    print('processing tuned model')
    tuned_history = tuned_model_1.fit(
        partial_train_data,
        partial_train_targets,
        batch_size=tuned_m_batch,
        epochs=num_epochs,
        validation_data=(val_data, val_targets),
        callbacks=[early_stopping]
    )
    print('processing hibrid model 1')
    h_history= hybrid_model.fit(
        partial_train_data,
        partial_train_targets,
        batch_size=32,
        epochs=num_epochs,
        validation_data=(val_data, val_targets),
        callbacks=[early_stopping]
    )
    # Evaluate the kfold results for BASELINE
    base_val_loss, base_val_accuracy = baseline_model.evaluate(val_data, val_targets, verbose=0)
    base_all_val_losses.append(base_val_loss)
    base_all_val_acc.append(base_val_accuracy)

    # Evaluate the kfold results for the tuned model
    tuned_val_loss, tuned_val_accuracy = tuned_model_1.evaluate(val_data, val_targets, verbose=0)
    tuned_all_val_losses.append(tuned_val_loss)
    tuned_all_val_acc.append(tuned_val_accuracy)

    # Evaluate the kfold results for the hibrid model 1
    res_val_loss, res_val_accuracy = hybrid_model.evaluate(val_data, val_targets, verbose=0)
    res_all_val_losses.append(res_val_loss)
    res_all_val_acc.append(res_val_accuracy)

In [ ]:
print("Baseline Model KF Results:")
print("-"*len("Baseline Model Results:"))
print("Avg val_acc: {}".format(np.mean(base_all_val_acc)))
print("Avg val_loss: {}".format(np.mean(base_all_val_losses)))
print()
print("Tuned Model KF Results:")
print("-"*len("Tuned Model KF Results:"))
print("Avg val_acc: {}".format(np.mean(tuned_all_val_acc)))
print("Avg val_loss: {}".format(np.mean(tuned_all_val_losses)))
print()
print("ResNet Model KF Results:")
print("-"*len("ResNetmodel KF Results:"))
print("Avg val_acc: {}".format(np.mean(res_all_val_acc)))
print("Avg val_loss: {}".format(np.mean(res_all_val_losses)))

In [ ]:
import matplotlib.pyplot as plt

epochs = range(1, num_epochs+1)

# Plotting the results of validation accuracy and loss for the baseline and tuned models
plt.subplot(2, 2, 1)
plt.plot(epochs, baseline_history.history['val_accuracy'], 'b', label='Baseline model')
plt.plot(epochs, tuned_history.history['val_accuracy'], 'r', label='Tuned model')
plt.title('Validation accuracy comparison')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(2, 2, 2)
plt.plot(epochs, baseline_history.history['val_loss'], 'b', label='Baseline model')
plt.plot(epochs, tuned_history.history['val_loss'], 'r', label='Tuned model')
plt.title('Validation loss comparison')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plotting the results of training accuracy and loss for the baseline and tuned models  
plt.subplot(2, 2, 3)
plt.plot(epochs, baseline_history.history['accuracy'], 'b', label='Baseline model')
plt.plot(epochs, tuned_history.history['accuracy'], 'r', label='Tuned model')
plt.title('Training accuracy comparison')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(2, 2, 4)
plt.plot(epochs, baseline_history.history['loss'], 'b', label='Baseline model')
plt.plot(epochs, tuned_history.history['loss'], 'r', label='Tuned model')
plt.title('Training loss comparison')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.subplots_adjust(hspace=0.4, wspace=0.4)
plt.show()

### 3.5 Analyze performance of fine-tuned model

# 4. Transfer Learning

### 4.1 New data split adapted to transfer learning

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.2, random_state=42, stratify=labels)

### 4.2 Transfer learning with VGG16

In [ ]:
# Import VGG16
vgg_model = VGG16(include_top=False, input_shape=(156, 156, 3))

# FReezing VGG16 layers
for layer in vgg_model.layers:
    layer.trainable = False

## adding "custom" layers

## Flatten layer
flat_1 = layers.Flatten()(vgg_model.layers[-1].output)

## Dense layers
dense_1 = layers.Dense(32, activation='relu')(flat_1)

#output layer with softmax 
output = layers.Dense(4, activation='softmax')(dense_1)

# define new model
tl_model = Model(inputs=vgg_model.inputs, outputs=output)

# summarize
tl_model.summary()
 
# compile model
tl_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Early stopping
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

# fit model
fitting = tl_model.fit(X_train, y_train_onehot, validation_data=(X_val, y_val_onehot), batch_size=64 ,epochs=25, verbose=1)

### 4.3 Analyze performance of transfer learning model